In [71]:
import fitz  # PyMuPDF
import cv2
import numpy as np
from PIL import Image, ImageDraw
import io

In [72]:
# === INPUT FILES ===
PDF_FILE = "Plans\C1.01.pdf"     # Your construction plan PDF
SYMBOL_IMAGE = "Symbols\sign-symbol.png"          # The dot symbol image
OUTPUT_PDF = "Annotated-Plans\C1.01_annotated.pdf"  # Output file name
threshold = 0.75                   # Lowered to improve symbol detection
dpi = 600                   # DPI for the output PDF

<>:2: SyntaxWarning: invalid escape sequence '\C'
<>:3: SyntaxWarning: invalid escape sequence '\s'
<>:4: SyntaxWarning: invalid escape sequence '\C'
<>:2: SyntaxWarning: invalid escape sequence '\C'
<>:3: SyntaxWarning: invalid escape sequence '\s'
<>:4: SyntaxWarning: invalid escape sequence '\C'
C:\Users\KaitlynHardy\AppData\Local\Temp\ipykernel_12660\3950573398.py:2: SyntaxWarning: invalid escape sequence '\C'
  PDF_FILE = "Plans\C1.01.pdf"     # Your construction plan PDF
C:\Users\KaitlynHardy\AppData\Local\Temp\ipykernel_12660\3950573398.py:3: SyntaxWarning: invalid escape sequence '\s'
  SYMBOL_IMAGE = "Symbols\sign-symbol.png"          # The dot symbol image
C:\Users\KaitlynHardy\AppData\Local\Temp\ipykernel_12660\3950573398.py:4: SyntaxWarning: invalid escape sequence '\C'
  OUTPUT_PDF = "Annotated-Plans\C1.01_annotated.pdf"  # Output file name


In [ ]:
# === Load symbol template ===
symbol_img = cv2.imread(symbol_path, cv2.IMREAD_GRAYSCALE)
symbol_img = cv2.resize(symbol_img, (15, 15), interpolation=cv2.INTER_AREA)
symbol_w, symbol_h = symbol_img.shape[::-1]

# === Open PDF ===
doc = fitz.open(PDF_FILE)
annotated_images = []

# === High-res matrix ===
zoom = dpi / 72
mat = fitz.Matrix(zoom, zoom)

# Increase the decompression bomb limit (use with caution)
from PIL import Image as PILImage
PILImage.MAX_IMAGE_PIXELS = None

# === Process each page ===
for page_num, page in enumerate(doc):
    pix = page.get_pixmap(matrix=mat)
    img_bytes = pix.tobytes("ppm")

    # Convert pixmap to OpenCV image
    pil_image = Image.open(io.BytesIO(img_bytes)).convert("RGB")
    cv_image = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)
    gray = cv2.cvtColor(cv_image, cv2.COLOR_BGR2GRAY)

    annotated = pil_image.copy()
    draw = ImageDraw.Draw(annotated)

    # Try multiple rotations
    for angle in range(0, 360, 30):
        M = cv2.getRotationMatrix2D((symbol_w/2, symbol_h/2), angle, 1)
        rotated_symbol = cv2.warpAffine(symbol_img, M, (symbol_w, symbol_h))

        res = cv2.matchTemplate(gray, rotated_symbol, cv2.TM_CCOEFF_NORMED)
        loc = np.where(res >= threshold)

        for pt in zip(*loc[::-1]):
            center_x = pt[0] + symbol_w // 2
            center_y = pt[1] + symbol_h // 2
            radius = int(max(symbol_w, symbol_h) / 2)
            draw.ellipse(
                (center_x - radius, center_y - radius,
                 center_x + radius, center_y + radius),
                outline="red", width=5
            )

    annotated_images.append(annotated)



In [74]:
# === Create final high-res PDF ===
out_doc = fitz.open()
for img in annotated_images:
    # Save PIL image to a bytes buffer as PNG
    img_buffer = io.BytesIO()
    img.save(img_buffer, format="PNG")
    img_buffer.seek(0)
    # Create Pixmap from PNG bytes
    pix = fitz.Pixmap(img_buffer)
    rect = fitz.Rect(0, 0, pix.width, pix.height)
    page = out_doc.new_page(width=pix.width, height=pix.height)
    page.insert_image(rect, pixmap=pix)

out_doc.save(OUTPUT_PDF)
print(f"✅ Annotated PDF saved as: {OUTPUT_PDF}")

✅ Annotated PDF saved as: Annotated-Plans\C1.01_annotated.pdf
